{1} DEAP. Distributed Evolutionary Algorithms in Python. Disponível em: https://deap.readthedocs.io/en/master/. Acesso em: 1 mar 2021.

In [ ]:
#[1] Importando pacotes e módulos

import random
import numpy

! pip install deap # Instalação de DEAP
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

     |████████████████████████████████| 160 kB 29.9 MB/s 


#**Seleção e Formulação do Problema** 

In [ ]:
#[2] Geração do grafo para o problema do caixeiro

# função graphTSP(numCities, minDist, maxDist)
# parâmetros: 
#   numCities: número de cities
#   minDist: menor valor de distância
#   maxDist: maior valor de distância
# retorno:
#   cities: grafo de cidades (Matriz numCities X numCities). As distância 
#   entre duas cidades são determinadas aleatoriamente entre minDist e maxDist
def graphTSP(numCities, minDist, maxDist):
  cities = numpy.zeros((numCities, numCities), dtype = int)
  for i in range(numCities):
    for j in range(numCities):
      if (j>i):
        cities[i, j] = random.randint(minDist, maxDist)
      elif (j<i):
        cities[i, j] = cities[j, i]
  return cities

numCities = 5     #  Número de cidade inicial

while(True):
  numCities = int(input('Digite o número de cidades: '))
  if (numCities > 4):
    break
  else:
    print('O número de cidades deve ser maior que 4!')

cities = graphTSP(numCities, 10, 100)
print('Grafo:\n', cities)

Digite o número de cidades: 10
Grafo:
 [[  0  53  74  81  12  73  44  36  56  72]
 [ 53   0  35  75  20  95  50  91  14  58]
 [ 74  35   0  16  39  97  62  45  48 100]
 [ 81  75  16   0  15  38  53  28  99  18]
 [ 12  20  39  15   0  78  11  79  31  87]
 [ 73  95  97  38  78   0  19  48  92  51]
 [ 44  50  62  53  11  19   0 100  29  33]
 [ 36  91  45  28  79  48 100   0  94  15]
 [ 56  14  48  99  31  92  29  94   0  11]
 [ 72  58 100  18  87  51  33  15  11   0]]


# **População e Indivíduos**

In [ ]:
#[3] Definição da Geração dos Indivíduos

creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # minimizar = peso negativo 
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# gerador de parâmetros 
toolbox.register("attr_int", random.randint, 0, numCities-1)

# define como os indivíduos/população é gerada
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, numCities)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#**Avaliação da Aptidão**

In [ ]:
#[4] Função para a avaliação da aptidão

# Função evalRoute(individual)
# parâmetros:
#   individual: uma rota
# retorno:
#   (cost, ): tupla contendo apenas o custo da rota (cost).
#             * precisa ser uma tupla devido a exigências do pacote DEAP 
def evalRoute(individual):
  cost = 0
  for i in range(1, len(individual)):
    if (individual.count(individual[i])>1):
      cost = cost + 1000000 # penalidade por repetir cidade
    cost = cost + cities[individual[i-1], individual[i]]        
  cost = cost + cities[individual[i],individual[0]]
  return (cost,)

#**Processamento do Algoritmo Genético**

> **Gerações**
* A cada iteração do algoritmo, um novo conjunto de indivíduos é gerado a partir da população anterior.
* Cada novo conjunto é chamado de “Geração”.
* Através da criação de uma grande quantidade de gerações que é possível obter resultados dos Algoritmos Genéticos.

>**Avaliação de Aptidão**
* A função de aptidão é aplicada ao fenótipo do indivíduo.

>**Seleção**
* Selecionar os indivíduos sobre os quais serão aplicados os operadores genéticos.
* Escolhe preferencialmente, embora não exclusivamente, indivíduos com maior aptidão. 
* Há diversas técnicas de seleção, entre elas há o método de seleção por Roleta e o método de seleção por Torneio.

>**Cruzamento**
* Também conhecida por Crossover ou Recombinação.
* Recombinação de características dos pais (Figura 2).
  * Permite que as próximas gerações herdem essas características.
* Escolhe dois indivíduos e troca trechos dos cromossomos entre eles.

>**Mutação**
* Introdução e manutenção da diversidade genética.
* Altera aleatoriamente um ou mais genes no cromossomo (Figura 3).




In [ ]:
#[5] Processamento do Algoritmo Genético

# Definindo avaliação de aptidão, seleção, cruzamento e mutação 
toolbox.register("evaluate", evalRoute)
toolbox.register("select", tools.selTournament, tournsize=3) # seleção por torneio
toolbox.register("mate", tools.cxOnePoint) # um ponto de cruzamento
toolbox.register("mutate", tools.mutUniformInt, low=0, up=numCities-1, indpb=0.05)

def main():
  print('Execução do algoritmo genético:')

  # random.seed(64)
  NGEN = 100     # número de gerações
  MU = 50        # tamanho da população
  LAMBDA = 100   # número de filhos gerados
  CXPB = 0.7     # probabilidade de cruzamento
  MUTPB = 0.3    # probabilidade de mutação
  
  pop = toolbox.population(n=MU)
  hof = tools.ParetoFront()
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", numpy.mean, axis=0)
  stats.register("std", numpy.std, axis=0)
  stats.register("min", numpy.min, axis=0)
  stats.register("max", numpy.max, axis=0)
  
  algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                            halloffame=hof)
      
  print('\nRota:', hof[0],'\nCusto:', evalRoute(hof[0])[0])
  return pop, stats, hof
                 
if __name__ == "__main__":
    main()

Execução do algoritmo genético:
gen	nevals	avg         	std               	min       	max       
0  	50    	[5580483.06]	[1312835.03206635]	[3000430.]	[8000540.]
1  	100   	[4340493.06]	[1193404.68264002]	[1000664.]	[7000297.]
2  	100   	[3460531.5] 	[853404.87015168] 	[1000664.]	[5000459.]
3  	100   	[3180513.42]	[993716.58832587] 	[1000664.]	[5000512.]
4  	100   	[2660543.5] 	[1012085.52424076]	[1000612.]	[5000619.]
5  	100   	[2260512.94]	[912275.37668567] 	[1000518.]	[4000391.]
6  	100   	[1940536.44]	[881045.53133593] 	[1000518.]	[4000422.]
7  	100   	[1560559.22]	[697341.35323476] 	[1000612.]	[3000568.]
8  	100   	[1220597.9] 	[501524.84535402] 	[1000612.]	[3000318.]
9  	100   	[1080609.52]	[439979.08780933] 	[1000531.]	[4000479.]
10 	100   	[1000610.98]	[4.03727631]      	[1000595.]	[1000612.]
11 	100   	[960609.62] 	[195961.95965931] 	[596.]    	[1000612.]
12 	100   	[940604.88] 	[237489.08536761] 	[596.]    	[1000612.]
13 	100   	[940596.8]  	[237487.04397874] 	[596.]    	[100